<a href="https://colab.research.google.com/github/ss1111119/ChillTrade/blob/main/ShouldISell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import textwrap
import google.generativeai as genai
import pandas as pd
from datetime import datetime, time
import talib as ta
import requests
from bs4 import BeautifulSoup
import numpy as np  # **新增部分**

# 顯示當前日期和時間
now = datetime.now()
print(f"當前日期和時間: {now.strftime('%Y-%m-%d %H:%M:%S')}")

# 設定參數
api_key = ''  # 請替換為您的 Gemini API Key，留空則不啟用
ticker = '2330.TW'  # 要分析的股票代碼
buy_price = None  # 您的購買價格，留空則忽略

# 使用 Yahoo Finance 抓取即時股價


def fetch_real_time_price(ticker):
    url = f'https://tw.stock.yahoo.com/quote/{ticker}'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        price_element = soup.select_one('.Fz\\(32px\\)')
        if price_element:
            try:
                current_price = float(
                    price_element.get_text().replace(',', ''))
                return current_price
            except ValueError:
                return None
        else:
            return None
    else:
        return None


# 在分析股票之前先抓取最新價格
current_price = fetch_real_time_price(ticker)
if current_price:
    print(f"{ticker} 的當前股價為: {current_price} TWD")
else:
    print("未能成功抓取股價")


# 檢查當前時間是否在市場交易時間內


def is_market_open():
    now = datetime.now()
    market_open_time = time(9, 0)  # 假設市場開盤時間為上午9點
    market_close_time = time(13, 30)  # 假設市場收盤時間為下午1點30分
    return market_open_time <= now.time() <= market_close_time

# 計算 MACD 指標


def calculate_macd(prices, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macd_signal, macd_diff = ta.MACD(
        prices, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macd_signal, macd_diff

# 計算 RSI 指標


def calculate_rsi(prices, timeperiod=14):
    return ta.RSI(prices, timeperiod)

# 計算布林帶（Bollinger Bands）


def calculate_bollinger_bands(prices, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0):
    """
    計算布林帶
    :param prices: 股票價格數據（例如收盤價）
    :param timeperiod: 計算移動平均線的周期，默認為20
    :param nbdevup: 上軌帶距離移動平均線的標準差倍數，默認為2
    :param nbdevdn: 下軌帶距離移動平均線的標準差倍數，默認為2
    :param matype: 移動平均類型，默認為0（簡單移動平均）
    :return: upperband, middleband, lowerband
    """
    upperband, middleband, lowerband = ta.BBANDS(
        prices, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    return upperband, middleband, lowerband

# 計算隨機震盪指標（Stochastic Oscillator）


def calculate_stochastic_oscillator(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3):
    """
    計算隨機震盪指標
    :param high: 股票最高價數據
    :param low: 股票最低價數據
    :param close: 股票收盤價數據
    :param fastk_period: 快速K線的周期，默認為14
    :param slowk_period: 慢速K線的周期，默認為3
    :param slowd_period: 慢速D線的周期，默認為3
    :return: slowk, slowd
    """
    slowk, slowd = ta.STOCH(
        high, low, close, fastk_period=fastk_period, slowk_period=slowk_period, slowd_period=slowd_period)
    return slowk, slowd

# 計算平均真實波幅（ATR）


def calculate_atr(high, low, close, timeperiod=14):
    """
    計算平均真實波幅（ATR）
    :param high: 股票最高價數據
    :param low: 股票最低價數據
    :param close: 股票收盤價數據
    :param timeperiod: 計算ATR的周期，默認為14
    :return: atr
    """
    atr = ta.ATR(high, low, close, timeperiod=timeperiod)
    return atr

# ***新增的趨勢指標計算函數***

# 計算 EMA（Exponential Moving Average，指數移動平均線）


def calculate_ema(prices, timeperiod=20):
    return ta.EMA(prices, timeperiod=timeperiod)

# 計算 SMA（Simple Moving Average，簡單移動平均線）


def calculate_sma(prices, timeperiod=50):
    return ta.SMA(prices, timeperiod=timeperiod)

# 計算 ADX（Average Directional Index，平均趨向指數）


def calculate_adx(high, low, close, timeperiod=14):
    return ta.ADX(high, low, close, timeperiod=timeperiod)

# **修正后的OBV计算函数**


def calculate_obv(close_prices, volumes):
    obv = np.zeros_like(close_prices)
    for i in range(1, len(close_prices)):
        if close_prices.iloc[i] > close_prices.iloc[i - 1]:  # 修改为使用 .iloc
            obv[i] = obv[i - 1] + volumes.iloc[i]  # 修改为使用 .iloc
        elif close_prices.iloc[i] < close_prices.iloc[i - 1]:  # 修改为使用 .iloc
            obv[i] = obv[i - 1] - volumes.iloc[i]  # 修改为使用 .iloc
        else:
            obv[i] = obv[i - 1]
    return obv


# **新增部分：計算 VWAP 指標**
def calculate_vwap(high, low, close, volume):
    typical_price = (high + low + close) / 3
    vwap = (typical_price * volume).cumsum() / volume.cumsum()
    return vwap


# 分析股票
def analyze_stock(ticker, buy_price):
    if not is_market_open():
        print("提醒：當前時間不在市場交易時間內，數據可能不完整或不準確。")

    stock = yf.Ticker(ticker)

    try:
        info = stock.info
        quote_type = info.get('quoteType', 'UNKNOWN')
    except Exception as e:
        print(f"從 Yahoo Finance 獲取數據時出現錯誤: {e}")
        info = {}
        quote_type = 'UNKNOWN'

    # 獲取股票或 ETF 的信息
    try:
        info = stock.info
        quote_type = info.get('quoteType', 'UNKNOWN')
    except Exception as e:
        print(f"從 Yahoo Finance 獲取數據時出現錯誤: {e}")
        info = {}
        quote_type = 'UNKNOWN'

    if quote_type == 'ETF':  # 判斷是否為 ETF
        return analyze_etf(stock, buy_price, info)  # 分析 ETF
    else:
        return analyze_equity(stock, buy_price)  # 分析股票

# 分析 ETF 的函數


def analyze_etf(stock, buy_price, info):
    # 使用 `info` 中的 ETF 特有資料
    aum = info.get('totalAssets', '無法取得')
    expense_ratio = info.get('expenseRatio', '無法取得')
    ytd_return = info.get('ytdReturn', '無法取得')
    three_year_return = info.get('threeYearAverageReturn', '無法取得')
    five_year_return = info.get('fiveYearAverageReturn', '無法取得')
    fifty_two_week_low = info.get('fiftyTwoWeekLow', '無法取得')
    fifty_two_week_high = info.get('fiftyTwoWeekHigh', '無法取得')
    fifty_day_avg = info.get('fiftyDayAverage', '無法取得')
    two_hundred_day_avg = info.get('twoHundredDayAverage', '無法取得')
    volume = info.get('volume', '無法取得')
    average_volume = info.get('averageVolume', '無法取得')
    beta = info.get('beta3Year', '無法取得')

    # 取得 ETF 的歷史價格數據和技術指標
    stock_data = stock.history(
        start="2021-01-01", end=datetime.now().strftime('%Y-%m-%d'))

    # 使用 'Adj Close' 列，如果不存在則使用 'Close'
    if 'Adj Close' not in stock_data.columns:
        stock_data['Adj Close'] = stock_data['Close']

    # 計算動量指標
    stock_data['upperband'], stock_data['middleband'], stock_data['lowerband'] = calculate_bollinger_bands(
        stock_data['Adj Close'])
    stock_data['slowk'], stock_data['slowd'] = calculate_stochastic_oscillator(
        stock_data['High'], stock_data['Low'], stock_data['Close'])
    stock_data['atr'] = calculate_atr(
        stock_data['High'], stock_data['Low'], stock_data['Close'])

    # 計算 MACD 和 RSI
    stock_data['macd'], stock_data['macd_signal'], stock_data['macd_diff'] = calculate_macd(
        stock_data['Adj Close'])
    stock_data['rsi'] = calculate_rsi(stock_data['Adj Close'])

    # ***新增的趨勢指標計算***
    stock_data['ema_20'] = calculate_ema(stock_data['Adj Close'])
    stock_data['sma_50'] = calculate_sma(stock_data['Adj Close'])
    stock_data['adx'] = calculate_adx(
        stock_data['High'], stock_data['Low'], stock_data['Close'])

    # 取得股息紀錄
    dividends = stock.dividends

    # 返回分析結果
    return {
        'aum': aum,
        'expense_ratio': expense_ratio,
        'ytd_return': ytd_return,
        'three_year_return': three_year_return,
        'five_year_return': five_year_return,
        'fifty_two_week_low': fifty_two_week_low,
        'fifty_two_week_high': fifty_two_week_high,
        'fifty_day_avg': fifty_day_avg,
        'two_hundred_day_avg': two_hundred_day_avg,
        'volume': volume,
        'average_volume': average_volume,
        'beta': beta,
        'dividends': dividends,
        'current_price': stock_data['Adj Close'].iloc[-1],
        'rsi': stock_data['rsi'].iloc[-1],
        'macd': stock_data['macd'].iloc[-1],
        'macd_signal': stock_data['macd_signal'].iloc[-1],
        'ema_20': stock_data['ema_20'].iloc[-1],  # ***新增的EMA指標結果***
        'sma_50': stock_data['sma_50'].iloc[-1],  # ***新增的SMA指標結果***
        'adx': stock_data['adx'].iloc[-1]         # ***新增的ADX指標結果***
    }

# 格式化 ETF 分析文本


def format_etf_analysis_text(result):
    def format_value(value, fmt="{:.2f}", default="無法取得"):
        if isinstance(value, (int, float)):
            return fmt.format(value)
        elif isinstance(value, str):
            return value
        else:
            return default

    analysis_text = f"""
    ETF 分析報告 ({ticker}):

    **基金基本資料:**
    - 基金規模 (AUM): {format_value(result['aum'], "{:,.0f}")} TWD
    - 費用率: {format_value(result['expense_ratio'], "{:.2f}")}%
    - 年初至今回報率 (YTD Return): {format_value(result['ytd_return'], "{:.2%}")}%
    - 三年平均回報率: {format_value(result['three_year_return'], "{:.2%}")}%
    - 五年平均回報率: {format_value(result['five_year_return'], "{:.2%}")}%

    **價格區間:**
    - 52周最低價: {format_value(result['fifty_two_week_low'])} TWD
    - 52周最高價: {format_value(result['fifty_two_week_high'])} TWD
    - 50天平均價: {format_value(result['fifty_day_avg'])} TWD
    - 200天平均價: {format_value(result['two_hundred_day_avg'])} TWD

    **技術指標:**
    - 當前價格: {format_value(result['current_price'])} TWD
    - 相對強弱指數 (RSI): {format_value(result['rsi'])}
    - MACD: {format_value(result['macd'])}
    - MACD Signal: {format_value(result['macd_signal'])}
    - EMA 20日: {format_value(result['ema_20'])} TWD  # ***新增的EMA結果***
    - SMA 50日: {format_value(result['sma_50'])} TWD  # ***新增的SMA結果***
    - ADX: {format_value(result['adx'])}             # ***新增的ADX結果***
    - 成交量: {format_value(result['volume'])}
    - 平均成交量: {format_value(result['average_volume'])}
    - 三年贝塔系数: {format_value(result['beta'])}

    **股息記錄:**
    - {result['dividends'].tail() if result['dividends']
       is not None else '無法取得'}
    """
    return textwrap.dedent(analysis_text).strip()

# 分析股票的函數


def analyze_equity(stock, buy_price):
    # 先抓取最新的股價
    current_price = fetch_real_time_price(ticker)

    # 取得歷史價格數據
    stock_data = stock.history(
        start="2021-01-01", end=datetime.now().strftime('%Y-%m-%d'))

    # 检查是否有 'Adj Close' 列，如果没有则使用 'Close'
    closing_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'

    # 如果成功获取到实时价格，使用实时价格替代历史数据中的最后一个收盘价
    if current_price:
        stock_data.loc[stock_data.index[-1], closing_column] = current_price
    else:
        current_price = stock_data[closing_column].iloc[-1]

    # 获取股票的相关信息
    try:
        info = stock.info
        quote_type = info.get('quoteType', 'UNKNOWN')
    except Exception as e:
        print(f"從 Yahoo Finance 獲取數據時出現錯誤: {e}")
        info = {}
        quote_type = 'UNKNOWN'

    # 取得股息和拆股信息
    dividends = stock.dividends
    splits = stock.splits

    # 取得財務報表
    balance_sheet = stock.balance_sheet
    income_statement = stock.financials
    cashflow = stock.cashflow

    # 提取資產負債表的主要項目
    total_assets = balance_sheet.loc['Total Assets'].iloc[0] if 'Total Assets' in balance_sheet.index else '無法取得'
    total_liabilities = balance_sheet.loc['Total Liab'].iloc[0] if 'Total Liab' in balance_sheet.index else '無法取得'
    total_equity = balance_sheet.loc['Total Stockholder Equity'].iloc[
        0] if 'Total Stockholder Equity' in balance_sheet.index else '無法取得'

    # 新增財務杠桿指標
    debt_to_equity = info.get('debtToEquity', '無法取得')
    interest_coverage = info.get('interestCoverage', '無法取得')

    # 取得估值指標
    price_to_book = info.get('priceToBook', '無法取得')
    price_to_sales = info.get('priceToSalesTrailing12Months', '無法取得')
    dividend_yield = info.get('dividendYield', '無法取得')

    # 獲取市場資本化、市盈率等數據
    market_cap = info.get('marketCap', '無法取得')
    pe_ratio = info.get('trailingPE', '無法取得')
    eps = info.get('trailingEps', '無法取得')
    industry = info.get('industry', '無法取得')
    description = info.get('longBusinessSummary', '無法取得')

    # 新增經營現金流和自由現金流部分
    operating_cash_flow = info.get('operatingCashflow', '無法取得')
    free_cashflow = info.get('freeCashflow', '無法取得')

    # 新增盈利能力指標計算部分
    revenue = info.get('totalRevenue')
    cost_of_goods_sold = info.get('costOfRevenue')
    operating_income = info.get('operatingIncome')
    net_income = info.get('netIncome')

    profitability_indicators = {}
    profitability_indicators['gross_margin'] = (
        revenue - cost_of_goods_sold) / revenue if revenue and cost_of_goods_sold else "無法計算"
    profitability_indicators['operating_margin'] = operating_income / \
        revenue if revenue and operating_income else "無法計算"
    profitability_indicators['net_profit_margin'] = net_income / \
        revenue if revenue and net_income else "無法計算"

    # 處理財務比率
    current_ratio = info.get('currentRatio', '無法取得')
    quick_ratio = info.get('quickRatio', '無法取得')
    profit_margin = info.get('profitMargins', '無法取得')

    # 獲取 ROA
    roa = info.get('returnOnAssets', '無法取得')

    # 處理財務數據異常
    try:
        recommendations = stock.recommendations
    except Exception as e:
        recommendations = "無法取得推薦數據"
        print(f"從 Yahoo Finance 獲取推薦數據時出現錯誤: {e}")

    # 取得自由現金流
    free_cashflow = info.get('freeCashflow')

    # 取得收益增長率
    earnings_growth = info.get('earningsGrowth')

    # 取得收入增長率
    revenue_growth = info.get('revenueGrowth')

    # 取得淨利潤率
    net_profit_margin = info.get('netProfitMargins')

    # 取得股東權益回報率 (ROE)
    roe = info.get('returnOnEquity')

    # 取得企業價值
    enterprise_value = info.get('enterpriseValue')

    # 取得每股淨資產 (BVPS)
    bvps = info.get('bookValue')

    # 取得分析師推薦
    try:
        recommendations = stock.recommendations
    except Exception as e:
        recommendations = "無法取得推薦數據"
        print(f"從 Yahoo Finance 獲取推薦數據時出現錯誤: {e}")

    # 取得貝塔係數
    beta = info.get('beta')

    # 取得股東信息
    try:
        major_holders = stock.major_holders
        institutional_holders = stock.institutional_holders
    except Exception as e:
        major_holders = "無法獲取主要股東信息"
        institutional_holders = "無法獲取機構持股信息"
        print(f"從 Yahoo Finance 獲取股東信息時出現錯誤: {e}")

    # 從財務報表中取得淨利潤和收入，並手動計算收入增長率和淨利潤率
    try:
        total_revenue_current = income_statement.loc['Total Revenue'].iloc[0]
        total_revenue_previous = income_statement.loc['Total Revenue'].iloc[1]
        revenue_growth = (total_revenue_current -
                          total_revenue_previous) / total_revenue_previous * 100

        net_income_current = income_statement.loc['Net Income'].iloc[0]
        net_income_previous = income_statement.loc['Net Income'].iloc[1]
        earnings_growth = (net_income_current -
                           net_income_previous) / net_income_previous * 100
        net_profit_margin = (net_income_current / total_revenue_current) * 100
    except KeyError:
        print("無法取得淨利潤或收入數據，無法計算淨利潤率或增長率。")
        revenue_growth = None
        earnings_growth = None
        net_profit_margin = None

    # 計算技術指標
    stock_data['macd'], stock_data['macd_signal'], stock_data['macd_diff'] = calculate_macd(
        stock_data[closing_column])
    delta = stock_data[closing_column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    stock_data['rsi'] = rsi

    # 新增動量指標
    stock_data['upperband'], stock_data['middleband'], stock_data['lowerband'] = calculate_bollinger_bands(
        stock_data[closing_column])
    stock_data['slowk'], stock_data['slowd'] = calculate_stochastic_oscillator(
        stock_data['High'], stock_data['Low'], stock_data['Close'])
    stock_data['atr'] = calculate_atr(
        stock_data['High'], stock_data['Low'], stock_data['Close'])

    # ***新增的趨勢指標計算***
    stock_data['ema_20'] = calculate_ema(stock_data[closing_column])
    stock_data['sma_50'] = calculate_sma(stock_data[closing_column])
    stock_data['adx'] = calculate_adx(
        stock_data['High'], stock_data['Low'], stock_data['Close'])

    # **新增部分：計算 OBV 和 VWAP**
    stock_data['obv'] = calculate_obv(
        stock_data['Close'], stock_data['Volume'])
    stock_data['vwap'] = calculate_vwap(
        stock_data['High'], stock_data['Low'], stock_data['Close'], stock_data['Volume'])

    # 如果 buy_price 為 None，跳過年化回報率的計算
    if buy_price:
        annual_return = (
            stock_data[closing_column].iloc[-1] / buy_price) ** (1 / 3) - 1
    else:
        annual_return = None

    result = {
        'current_price': current_price,
        'rsi': stock_data['rsi'].iloc[-1],
        'macd': stock_data['macd'].iloc[-1],
        'macd_signal': stock_data['macd_signal'].iloc[-1],
        'annual_return': annual_return * 100 if annual_return is not None else "無法計算",
        'price_to_book': price_to_book,
        'price_to_sales': price_to_sales,
        'dividend_yield': dividend_yield * 100 if dividend_yield != '無法取得' else '無法取得',
        'buy_price': buy_price,
        'dividends': dividends,
        'splits': splits,
        'balance_sheet': balance_sheet,
        'total_assets': total_assets,
        'total_liabilities': total_liabilities,
        'total_equity': total_equity,
        'income_statement': income_statement,
        'cashflow': cashflow,
        'market_cap': market_cap,
        'pe_ratio': pe_ratio,
        'eps': eps,
        'industry': industry,
        'description': description,
        'recommendations': recommendations,
        'beta': beta,
        'current_ratio': current_ratio,
        'quick_ratio': quick_ratio,
        'profit_margin': profit_margin,
        'free_cashflow': free_cashflow,
        'operating_cash_flow': operating_cash_flow,
        'earnings_growth': earnings_growth,
        'revenue_growth': revenue_growth,
        'net_profit_margin': net_profit_margin,
        'roe': roe,
        'enterprise_value': enterprise_value,
        'bvps': bvps,
        'major_holders': major_holders,
        'institutional_holders': institutional_holders,
        'upperband': stock_data['upperband'].iloc[-1],
        'middleband': stock_data['middleband'].iloc[-1],
        'lowerband': stock_data['lowerband'].iloc[-1],
        'slowk': stock_data['slowk'].iloc[-1],
        'slowd': stock_data['slowd'].iloc[-1],
        'atr': stock_data['atr'].iloc[-1],
        'ema_20': stock_data['ema_20'].iloc[-1],
        'sma_50': stock_data['sma_50'].iloc[-1],
        'adx': stock_data['adx'].iloc[-1],
        'obv': stock_data['obv'].iloc[-1],
        'vwap': stock_data['vwap'].iloc[-1],
        'gross_margin': profitability_indicators['gross_margin'],
        'operating_margin': profitability_indicators['operating_margin'],
        'net_profit_margin': profitability_indicators['net_profit_margin'],
        'debt_to_equity': debt_to_equity,
        'interest_coverage': interest_coverage,
        'roa': roa,
    }

    return result


# 格式化股票分析文本


def format_analysis_text(result):
    def format_value(value, fmt="{:.2f}", default="無法取得"):
        if isinstance(value, (int, float)):
            return fmt.format(value)
        elif isinstance(value, str):
            return value
        else:
            return default

    def format_dataframe(df, default="無法取得"):
        if df is None or df.empty:
            return default
        else:
            return df.tail(5).to_string()  # 只顯示最近5行，防止顯示過長

    analysis_text = f"""
    股票分析報告 ({ticker}):

    **股價和技術分析:**
    - 當前價格: {format_value(result['current_price'])} TWD
    - 購買價格: {format_value(result['buy_price']) if result['buy_price'] is not None else '無法取得'} TWD
    - 年化回報率: {format_value(result['annual_return'])}%
    - 相對強弱指數 (RSI): {format_value(result['rsi'])}
    - MACD: {format_value(result['macd'])}
    - MACD Signal: {format_value(result['macd_signal'])}

    **布林帶指標:**
    - 上軌帶: {format_value(result['upperband'])} TWD
    - 中軌帶: {format_value(result['middleband'])} TWD
    - 下軌帶: {format_value(result['lowerband'])} TWD

    **隨機震盪指標:**
    - 慢速K線: {format_value(result['slowk'])}
    - 慢速D線: {format_value(result['slowd'])}

    **平均真實波幅 (ATR):**
    - ATR: {format_value(result['atr'])}

    **趨勢指標:**
    - EMA 20日: {format_value(result['ema_20'])} TWD
    - SMA 50日: {format_value(result['sma_50'])} TWD
    - ADX: {format_value(result['adx'])}

    **成交量指標:**
    - OBV: {format_value(result['obv'])}
    - VWAP: {format_value(result['vwap'])} TWD

    **公司信息:**
    - 市場資本化: {format_value(result['market_cap'], "{:,.0f}")} TWD
    - 市盈率: {result.get('pe_ratio', '無法取得')}
    - 每股收益 (EPS): {result.get('eps', '無法取得')}
    - 行業: {result.get('industry', '無法取得')}
    - 公司描述: {result.get('description', '無法取得')}

    **估值指標:**
    - 市净率 (P/B): {format_value(result['price_to_book'])}
    - 市销率 (P/S): {format_value(result['price_to_sales'])}
    - 股息率 (Dividend Yield): {format_value(result['dividend_yield'])}%

    **財務比率:**
    - 當前比率: {result.get('current_ratio', '無法取得')}
    - 速動比率: {result.get('quick_ratio', '無法取得')}
    - 利潤率: {result.get('profit_margin', '無法取得')}%

    **現金流指標:**
    - 經營現金流: {format_value(result['operating_cash_flow'])} TWD
    - 自由現金流: {format_value(result['free_cashflow'])} TWD

    **財務杠桿指標（Leverage Indicators）:**
    - 負債權益比率 (Debt to Equity Ratio): {format_value(result['debt_to_equity'])}
    - 利息保障倍數 (Interest Coverage Ratio): {format_value(result['interest_coverage'])}

    **增長率分析:**
    - 收益增長率: {result.get('earnings_growth', '無法取得')}%
    - 收入增長率: {result.get('revenue_growth', '無法取得')}%

    **盈利能力指標 (Profitability Indicators):**
    - 毛利率 (Gross Margin): {format_value(result['gross_margin'])}
    - 營業利潤率 (Operating Margin): {format_value(result['operating_margin'])}
    - 淨利潤率 (Net Profit Margin): {format_value(result['net_profit_margin'])}

    **資產回報率 (ROA):**
    - {format_value(result['roa'], "{:.2%}")}

    **股東權益回報率 (ROE):**
    - {format_value(result['roe'], "{:.2%}")}

    **企業價值 (EV):**
    - {format_value(result['enterprise_value'], "{:,.0f}")} TWD

    **每股淨資產 (BVPS):**
    - {result.get('bvps', '無法取得')}

    **資產負債表摘要:**
    - 總資產: {format_value(result['total_assets'], "{:,.0f}")} TWD
    - 總負債: {format_value(result['total_liabilities'], "{:,.0f}")} TWD
    - 股東權益總額: {format_value(result['total_equity'], "{:,.0f}")} TWD

    **財務報表摘要:**
    - 資產負債表: {result['balance_sheet'].head() if result['balance_sheet'] is not None else '無法取得'}
    - 損益表: {result['income_statement'].head() if result['income_statement'] is not None else '無法取得'}
    - 現金流量表: {result['cashflow'].head() if result['cashflow'] is not None else '無法取得'}

    **股息和拆股信息:**
    - 股息: {result['dividends'].tail() if result['dividends'] is not None else '無法取得'}
    - 拆股: {result['splits']}

    **分析師推薦:**
    - {result['recommendations'].tail() if result['recommendations']
       is not None else '無法取得'}

    **股東信息:**
    - 主要持股人: {result.get('major_holders', '無法取得')}
    - 機構持股人: {result.get('institutional_holders', '無法取得')}

    **貝塔係數 (Beta):**
    - {result.get('beta', '無法取得')}
    """
    return textwrap.dedent(analysis_text).strip()


# 使用 Gemini API 生成報告


def generate_stock_analysis_with_gemini(api_key, analysis_text):
    if not api_key:
        return "Gemini API Key 未提供，將跳過 AI 生成的分析報告部分。"

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        full_prompt = (
      f"你是一個專業股票分析師，根據以下提供的資料，請為投資者制定當前最適合的投資策略，並逐項分析以下技術指標和財務指標的意義："
      f"相對強弱指數（RSI）、MACD、布林帶（Bollinger Bands）、成交量指標（如OBV和VWAP）、財務比率（如市盈率、股息率）、以及盈利能力指標（如毛利率、營業利潤率）。"
      f"特別要注意使用者的購入價格，根據當前價格提供止損建議。目前價格是應該賣出、持有還是繼續買入，並針對不同的投資情境（如短期與長期）提出具體的下一步行動建議（如是否應調整倉位、關注其他市場趨勢）。"
      f"在分析中請考慮當前的市場環境和經濟形勢。總結部分請至少300字，並結合上述分析給出整體投資建議，使用繁體中文回答：\n\n{analysis_text}"
  )


        response = model.generate_content(contents=[full_prompt])
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        return f"無法生成報告: {e}"


# 主程序
result = analyze_stock(ticker, buy_price)
if result:
    if 'aum' in result:  # 如果分析結果中包含 AUM，則表示是 ETF 分析
        analysis_text = format_etf_analysis_text(result)
    else:
        analysis_text = format_analysis_text(result)

    if api_key:
        gemini_analysis = generate_stock_analysis_with_gemini(
            api_key, analysis_text)
        full_report = (
            f"{analysis_text}\n\n"
            f"Gemini API 生成的分析報告:\n{gemini_analysis}"
        ) if gemini_analysis else "無法生成 Gemini 分析報告。"
    else:
        full_report = analysis_text

    print(full_report)